In [16]:
#<陳羽暉, 111753156>

import numpy as np
import math

# Read file
file = open("pam250.txt", "r")
data = file.readlines()
file.close()

pam250 = np.zeros((24, 24))
for i in range(10, 34):
    s = data[i].rstrip()
    rowData = s.split()
    del rowData[0]
    for j in range(len(rowData)):
        pam250[i - 10][j] = int(rowData[j])
aminoAcid = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'B', 'Z', 'X', '*']

file = open('test.fasta', 'r')
ls = []
name = []
for line in file:
    if line.startswith('>'):
        name.append(line)
    if not line.startswith('>'):
        ls.append(line.replace('\n', ''))
file.close()

#Caculate the dynamic programming
dp_score = np.zeros((len(ls[0]) + 1, len(ls[1]) + 1))
dp_direction = np.zeros((len(ls[0]) + 1, len(ls[1]) + 1))

direction = ['Left', 'Upper Left', 'Up']

dp_score[0][0] = 1
for i in range(1, len(ls[0]) + 1):
    amino = ls[0][i - 1]
    dp_score[i][0] = dp_score[i-1][0] + (-8)
for j in range(1, len(ls[1]) + 1):
    amino = ls[1][j - 1]
    dp_score[0][j] = dp_score[0][j-1] + (-8)

for i in range(1, len(ls[0]) + 1):
    for j in range(1, len(ls[1]) + 1):
        aminoIndex1 = aminoAcid.index(ls[0][i - 1])
        aminoIndex2 = aminoAcid.index(ls[1][j - 1])
        left = dp_score[i][j-1] + (-8)
        upperLeft = dp_score[i-1][j-1] + pam250[aminoIndex1][aminoIndex2]
        up = dp_score[i][j-1] + (-8)
        if (max(upperLeft, left, up) == left):
            dp_score[i][j] = left
            dp_direction[i][j] = 1
        elif (max(upperLeft, left, up) == upperLeft):
            dp_score[i][j] = upperLeft
            dp_direction[i][j] = 2
        elif (max(upperLeft, left, up) == up):
            dp_score[i][j] = up
            dp_direction[i][j] = 3
            
dp_score_local = np.zeros((len(ls[0]) + 1, len(ls[1]) + 1))
dp_direction_local = np.zeros((len(ls[0]) + 1, len(ls[1]) + 1))

direction = ['Left', 'Upper Left', 'Up']

dp_score_local[0][0] = 1
for i in range(1, len(ls[0]) + 1):
    amino = ls[0][i - 1]
    dp_score_local[i][0] = dp_score_local[i-1][0] + (-8)
    if dp_score_local[i][0] < 0:
        dp_score_local[i][0] = 0
for j in range(1, len(ls[1]) + 1):
    amino = ls[1][j - 1]
    dp_score_local[0][j] = dp_score_local[0][j-1] + (-8)
    if dp_score_local[0][j] < 0:
        dp_score_local[0][j] = 0

for i in range(1, len(ls[0]) + 1):
    for j in range(1, len(ls[1]) + 1):
        aminoIndex1 = aminoAcid.index(ls[0][i - 1])
        aminoIndex2 = aminoAcid.index(ls[1][j - 1])
        left = dp_score_local[i][j-1] + (-8)
        upperLeft = dp_score_local[i-1][j-1] + pam250[aminoIndex1][aminoIndex2]
        up = dp_score_local[i][j-1] + (-8)
        if (max(upperLeft, left, up) < 0):
            dp_score_local[i][j] = 0
        elif (max(upperLeft, left, up) == left):
            dp_score_local[i][j] = left
            dp_direction_local[i][j] = 1
        elif (max(upperLeft, left, up) == upperLeft):
            dp_score_local[i][j] = upperLeft
            dp_direction_local[i][j] = 2
        elif (max(upperLeft, left, up) == up):
            dp_score_local[i][j] = up
            dp_direction_local[i][j] = 3

#for i in range(len(ls[0]) + 1):
#    for j in range(len(ls[1]) + 1):
#        print(i, j, ":", dp_score_local[i][j], end=" ")
#    print('\n')
            
#Choose global or local alignment and trace back, then save the result to fasta
while(True):
    choose = str(input('global or local: '))
    if choose == 'global' or choose == 'local':
        break
    else:
        print('Please enter again.\n')

if choose == 'global':
    aminoAcid1 = []
    aminoAcid2 = []
    row = len(ls[0])
    column = len(ls[1])

    traceback = dp_direction[row][column]

    while (traceback != 0):
        if traceback == 3:
            aminoAcid1.append(ls[0][row-1])
            aminoAcid2.append('-')
            row -= 1
        elif traceback == 2:
            aminoAcid1.append(ls[0][row-1])
            aminoAcid2.append(ls[1][column-1])
            row -= 1
            column -= 1
        elif traceback == 1:
            aminoAcid1.append('-')
            aminoAcid2.append(ls[1][column-1])
            column -= 1
        traceback = dp_direction[row][column]

    aminoAcid1.reverse()
    aminoAcid2.reverse()
    result = [aminoAcid1, aminoAcid2]

    file = open('global.fasta', 'w')
    lines = []
    for i in range(len(result)):
        lines.append(name[i])
        a = ''.join(result[i]) + "\n"
        lines.append(a)
    file.writelines(lines)
    file.close()


elif (choose == 'local'):
    maxScore = 0
    for i in range(len(ls[0]) + 1):
        for j in range(len(ls[1]) + 1):
            if dp_score_local[i][j] > maxScore:
                maxScore = dp_score_local[i][j]
    maxIndex = []
    for i in range(len(ls[0]) + 1):
        for j in range(len(ls[1]) + 1):
            if dp_score_local[i][j] == maxScore:
                maxIndex.append((i, j))
    #print(maxIndex)
    result = []
    for i in range(len(maxIndex)):
        aminoAcid1 = []
        aminoAcid2 = []
        (row, column) = maxIndex[i]

        traceback = dp_direction_local[row][column]

        while (traceback != 0):
            if traceback == 3:
                aminoAcid1.append(ls[0][row-1])
                aminoAcid2.append('-')
                row -= 1
            elif traceback == 2:
                aminoAcid1.append(ls[0][row-1])
                aminoAcid2.append(ls[1][column-1])
                row -= 1
                column -= 1
            elif traceback == 1:
                aminoAcid1.append('-')
                aminoAcid2.append(ls[1][column-1])
                column -= 1
            traceback = dp_direction_local[row][column]
        aminoAcid1.reverse()
        aminoAcid2.reverse()
        result.append(aminoAcid1)
        result.append(aminoAcid2)
    
    finalResult = []
    maxLen = 0
    for i in range(len(result)):
        if len(result[i]) > maxLen:
            finalResult = []
            finalResult.append(result[i])
            maxLen = len(result[i])
        elif len(result[i]) == maxLen:
            finalResult.append(result[i])
    
    file = open('local.fasta', 'w')
    lines = []
    for i in range(0, len(finalResult), 2):
        lines.append(name[0])
        a = ''.join(finalResult[i]) + "\n"
        lines.append(a)
        lines.append(name[1])
        b = ''.join(finalResult[i+1]) + "\n"
        lines.append(b)
    file.writelines(lines)
    file.close()

global or local: local
